# Final assignment

In [1]:
library(data.table)
library(survey)

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: 'survey'


The following object is masked from 'package:graphics':

    dotchart




# Drug use difference by gender

In [3]:
sdf = fread('school-data.csv')
sdf[, f1 := length(unique(id_school)) / total_schools]
sdf[, f2 := .N / total_students, id_school]

In [4]:
length(unique(sdf$id_student))

[1] 450

In [5]:
prop.table(table(sdf$drug))


        0         1 
0.6466667 0.3533333 

In [6]:
sdf[, w := 1/(f1*f2)]
sum(sdf$w)

[1] 70200

In [9]:
library(survey)
design = svydesign(id = ~ id_school, data = sdf, probs = ~f1+f2, fpc = ~total_schools)

In [10]:
# t-test
ttest_result = svyttest(drug ~ female, design)
ttest_result


	Design-based t-test

data:  drug ~ female
t = -1.8033, df = 13, p-value = 0.09456
alternative hypothesis: true difference in mean is not equal to 0
95 percent confidence interval:
 -0.25805748  0.02324957
sample estimates:
difference in mean 
         -0.117404 


In [12]:
design2 = svydesign(id = ~ 1, data = sdf, probs = ~f1+f2)
ttest_result = svyttest(drug ~ female, design2)
ttest_result


	Design-based t-test

data:  drug ~ female
t = -2.3399, df = 448, p-value = 0.01973
alternative hypothesis: true difference in mean is not equal to 0
95 percent confidence interval:
 -0.21601016 -0.01879775
sample estimates:
difference in mean 
         -0.117404 


# PPS sampling

In [13]:
library(data.table)

svalues = seq(6, 310, 61)
block = 1:10
units = c(32, 18, 48, 15, 37, 26, 12, 45, 46, 21)

dat = data.table(block, units)

dat[, units_cum := cumsum(units)]
dat[, start := shift(units_cum, fill=0)]

In [14]:
svalues

[1]   6  67 128 189 250

In [15]:
dat

block,units,units_cum,start
<int>,<dbl>,<dbl>,<dbl>
1,32,32,0
2,18,50,32
3,48,98,50
4,15,113,98
5,37,150,113
6,26,176,150
7,12,188,176
8,45,233,188
9,46,279,233


In [359]:
selected_blocks = sapply(svalues, function(x) {
   dat[block == which(start < x & units_cum >= x), block]

})

sblocks = dat[block %in% selected_blocks]
sblocks

block,units,units_cum,start
<int>,<dbl>,<dbl>,<dbl>
1,32,32,0
3,48,98,50
5,37,150,113
8,45,233,188
9,46,279,233
